<a href="https://colab.research.google.com/github/Kondwani7/Pyspark-basics/blob/main/Pyspark_basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [63]:
from google.colab import drive
drive.mount('/content/drive', force_remount= True)

Mounted at /content/drive


In [3]:
!pip install pyspark

     |████████████████████████████████| 281.3 MB 40 kB/s 
     |████████████████████████████████| 198 kB 41.9 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=6bd00e19b1aedf82f81f571e8c967db32595e7f720a4195f63dd2cf1e9039b58
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


In [2]:
!pip install findspark

In [64]:
import pyspark
import findspark

In [65]:
sc = pyspark.SparkContext(appName='maps_lazy_evaluation')

ValueError: ignored

In [6]:
log_of_songs =[
               'I\'m sipping tea in your hood',
               'bedrock',
               'Party',
               'God\'s plan',
               'all of me'
]

Preprocess the data

In [7]:
distributed_songs = sc.parallelize(log_of_songs)

In [8]:
#convert all songs to lower case
def song_lowercase_convertor(song):
  return song.lower()


In [9]:
song_lowercase_convertor('This right heere I my time to partY')

'this right heere i my time to party'

In [10]:
distributed_songs.map(song_lowercase_convertor)

PythonRDD[1] at RDD at PythonRDD.scala:53

In [11]:
#despite the preprocessing, spark kept a copy of the original data
distributed_songs.collect()

["I'm sipping tea in your hood", 'bedrock', 'Party', "God's plan", 'all of me']

In [12]:
#our target preprocessed data
distributed_songs.map(lambda song: song.lower()).collect()

["i'm sipping tea in your hood", 'bedrock', 'party', "god's plan", 'all of me']

#Working with different Data Formats

In [13]:
from pyspark import SparkConf
from pyspark.sql import SparkSession

In [14]:
spark = SparkSession \
      .builder \
      .appName('Python Spark SQL data formats') \
      .getOrCreate()


In [15]:
#examine session change
spark.sparkContext.getConf().getAll()


[('spark.app.startTime', '1639649834643'),
 ('spark.driver.port', '43787'),
 ('spark.app.name', 'maps_lazy_evaluation'),
 ('spark.rdd.compress', 'True'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.master', 'local[*]'),
 ('spark.submit.pyFiles', ''),
 ('spark.executor.id', 'driver'),
 ('spark.submit.deployMode', 'client'),
 ('spark.driver.host', 'd2d100ef26f0'),
 ('spark.sql.warehouse.dir', 'file:/content/spark-warehouse'),
 ('spark.app.id', 'local-1639649836625'),
 ('spark.ui.showConsoleProgress', 'true')]

In [16]:
#info on the context created
spark

json data

In [17]:
json_path = '/content/drive/MyDrive/practice datasets/sparkify_log_small.json'
user_log = spark.read.json(json_path)

In [19]:
user_log.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [20]:
user_log.describe()

DataFrame[summary: string, artist: string, auth: string, firstName: string, gender: string, itemInSession: string, lastName: string, length: string, level: string, location: string, method: string, page: string, registration: string, sessionId: string, song: string, status: string, ts: string, userAgent: string, userId: string]

In [21]:
user_log.show(n=1)

+-------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-------------+---------+--------------------+------+-------------+--------------------+------+
|       artist|     auth|firstName|gender|itemInSession|lastName|   length|level|            location|method|    page| registration|sessionId|                song|status|           ts|           userAgent|userId|
+-------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-------------+---------+--------------------+------+-------------+--------------------+------+
|Showaddywaddy|Logged In|  Kenneth|     M|          112|Matthews|232.93342| paid|Charlotte-Concord...|   PUT|NextSong|1509380319284|     5132|Christmas Tears W...|   200|1513720872284|"Mozilla/5.0 (Win...|  1046|
+-------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-------------+---------+------

In [22]:
#show 7 rows in a structured format
user_log.take(7)

[Row(artist='Showaddywaddy', auth='Logged In', firstName='Kenneth', gender='M', itemInSession=112, lastName='Matthews', length=232.93342, level='paid', location='Charlotte-Concord-Gastonia, NC-SC', method='PUT', page='NextSong', registration=1509380319284, sessionId=5132, song='Christmas Tears Will Fall', status=200, ts=1513720872284, userAgent='"Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36"', userId='1046'),
 Row(artist='Lily Allen', auth='Logged In', firstName='Elizabeth', gender='F', itemInSession=7, lastName='Chase', length=195.23873, level='free', location='Shreveport-Bossier City, LA', method='PUT', page='NextSong', registration=1512718541284, sessionId=5027, song='Cheryl Tweedy', status=200, ts=1513720878284, userAgent='"Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"', userId='1000'),
 Row(artist='Cobra Starship Featuring Leighton Meester', auth='Logged In', firstNa

CSV file

In [23]:
out_path = '/content/drive/MyDrive/practice datasets/purchase data.csv'

In [24]:
user_log_2 = spark.read.csv(out_path, header=True)

In [51]:
user_log_2.printSchema()

root
 |-- ID: string (nullable = true)
 |-- Day: string (nullable = true)
 |-- Incidence: string (nullable = true)
 |-- Brand: string (nullable = true)
 |-- Quantity: string (nullable = true)
 |-- Last_Inc_Brand: string (nullable = true)
 |-- Last_Inc_Quantity: string (nullable = true)
 |-- Price_1: string (nullable = true)
 |-- Price_2: string (nullable = true)
 |-- Price_3: string (nullable = true)
 |-- Price_4: string (nullable = true)
 |-- Price_5: string (nullable = true)
 |-- Promotion_1: string (nullable = true)
 |-- Promotion_2: string (nullable = true)
 |-- Promotion_3: string (nullable = true)
 |-- Promotion_4: string (nullable = true)
 |-- Promotion_5: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Marital status: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- Education: string (nullable = true)
 |-- Income: string (nullable = true)
 |-- Occupation: string (nullable = true)
 |-- Settlement size: string (nullable = true)



In [52]:
user_log_2.take(5)

[Row(ID='200000001', Day='1', Incidence='0', Brand='0', Quantity='0', Last_Inc_Brand='0', Last_Inc_Quantity='0', Price_1='1.59', Price_2='1.87', Price_3='2.01', Price_4='2.09', Price_5='2.66', Promotion_1='0', Promotion_2='1', Promotion_3='0', Promotion_4='0', Promotion_5='0', Sex='0', Marital status='0', Age='47', Education='1', Income='110866', Occupation='1', Settlement size='0'),
 Row(ID='200000001', Day='11', Incidence='0', Brand='0', Quantity='0', Last_Inc_Brand='0', Last_Inc_Quantity='0', Price_1='1.51', Price_2='1.89', Price_3='1.99', Price_4='2.09', Price_5='2.66', Promotion_1='0', Promotion_2='0', Promotion_3='0', Promotion_4='0', Promotion_5='0', Sex='0', Marital status='0', Age='47', Education='1', Income='110866', Occupation='1', Settlement size='0'),
 Row(ID='200000001', Day='12', Incidence='0', Brand='0', Quantity='0', Last_Inc_Brand='0', Last_Inc_Quantity='0', Price_1='1.51', Price_2='1.89', Price_3='1.99', Price_4='2.09', Price_5='2.66', Promotion_1='0', Promotion_2='0

In [54]:
user_log_2.select('Price_1').show()

+-------+
|Price_1|
+-------+
|   1.59|
|   1.51|
|   1.51|
|   1.52|
|   1.52|
|    1.5|
|    1.5|
|    1.5|
|   1.35|
|   1.35|
|   1.35|
|   1.39|
|   1.39|
|   1.36|
|   1.47|
|   1.47|
|   1.39|
|   1.39|
|   1.47|
|   1.47|
+-------+
only showing top 20 rows



#Data Wrangling

In [25]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import desc
from pyspark.sql.functions import asc
from pyspark.sql.functions import sum as Fsum

import datetime

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [26]:
spark = SparkSession \
    .builder \
    .appName("Pyspark Data Wrangler") \
    .getOrCreate()

In [27]:
# show the data types
user_log.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [28]:
#print te first 3 rows
user_log.take(5)

[Row(artist='Showaddywaddy', auth='Logged In', firstName='Kenneth', gender='M', itemInSession=112, lastName='Matthews', length=232.93342, level='paid', location='Charlotte-Concord-Gastonia, NC-SC', method='PUT', page='NextSong', registration=1509380319284, sessionId=5132, song='Christmas Tears Will Fall', status=200, ts=1513720872284, userAgent='"Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36"', userId='1046'),
 Row(artist='Lily Allen', auth='Logged In', firstName='Elizabeth', gender='F', itemInSession=7, lastName='Chase', length=195.23873, level='free', location='Shreveport-Bossier City, LA', method='PUT', page='NextSong', registration=1512718541284, sessionId=5027, song='Cheryl Tweedy', status=200, ts=1513720878284, userAgent='"Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"', userId='1000'),
 Row(artist='Cobra Starship Featuring Leighton Meester', auth='Logged In', firstNa

In [29]:
#10,000 rows
user_log.count()

10000

In [30]:
#print artist names
user_log.select('artist').show()

+--------------------+
|              artist|
+--------------------+
|       Showaddywaddy|
|          Lily Allen|
|Cobra Starship Fe...|
|          Alex Smoke|
|                null|
|                null|
|              Redman|
|     Ulrich Schnauss|
|                null|
|                null|
|               Jay-Z|
|         Evanescence|
|     Scissor Sisters|
|        3 Doors Down|
|       George Younce|
|              Aly-Us|
|                null|
|            BjÃÂ¶rk|
|      David Bromberg|
|          Nickelback|
+--------------------+
only showing top 20 rows



In [31]:
#print artist names without duplicates
user_log.select("artist").dropDuplicates().sort("artist").show()


+--------------------+
|              artist|
+--------------------+
|                null|
|                 !!!|
|      10_000 Maniacs|
|           12 Stones|
|               1990s|
|      22 Pistepirkko|
|                2Mex|
|        3 Doors Down|
|                 311|
|             33Miles|
|       36 Crazyfists|
|                 3LW|
|               3OH!3|
|            3rd Bass|
|           3rd Force|
|             50 Cent|
|50 Cent / Lloyd B...|
|   59 Times the Pain|
|            9 Lazy 9|
|                 911|
+--------------------+
only showing top 20 rows



In [87]:
user_log.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)
 |-- hour: string (nullable = true)



counting stats by the hour

In [32]:
get_hour = udf(lambda x: datetime.datetime.fromtimestamp(x/ 10000.0).hour)


In [33]:
user_log = user_log.withColumn("hour", get_hour(user_log.ts))


In [34]:
user_log.take(5)

[Row(artist='Showaddywaddy', auth='Logged In', firstName='Kenneth', gender='M', itemInSession=112, lastName='Matthews', length=232.93342, level='paid', location='Charlotte-Concord-Gastonia, NC-SC', method='PUT', page='NextSong', registration=1509380319284, sessionId=5132, song='Christmas Tears Will Fall', status=200, ts=1513720872284, userAgent='"Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36"', userId='1046', hour='23'),
 Row(artist='Lily Allen', auth='Logged In', firstName='Elizabeth', gender='F', itemInSession=7, lastName='Chase', length=195.23873, level='free', location='Shreveport-Bossier City, LA', method='PUT', page='NextSong', registration=1512718541284, sessionId=5027, song='Cheryl Tweedy', status=200, ts=1513720878284, userAgent='"Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"', userId='1000', hour='23'),
 Row(artist='Cobra Starship Featuring Leighton Meester', aut

In [35]:
user_log.head(7)

[Row(artist='Showaddywaddy', auth='Logged In', firstName='Kenneth', gender='M', itemInSession=112, lastName='Matthews', length=232.93342, level='paid', location='Charlotte-Concord-Gastonia, NC-SC', method='PUT', page='NextSong', registration=1509380319284, sessionId=5132, song='Christmas Tears Will Fall', status=200, ts=1513720872284, userAgent='"Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36"', userId='1046', hour='23'),
 Row(artist='Lily Allen', auth='Logged In', firstName='Elizabeth', gender='F', itemInSession=7, lastName='Chase', length=195.23873, level='free', location='Shreveport-Bossier City, LA', method='PUT', page='NextSong', registration=1512718541284, sessionId=5027, song='Cheryl Tweedy', status=200, ts=1513720878284, userAgent='"Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"', userId='1000', hour='23'),
 Row(artist='Cobra Starship Featuring Leighton Meester', aut

In [36]:
#songs in a hour
songs_in_hour = user_log.filter(user_log.page == "NextSong").groupby(user_log.hour).count().orderBy(user_log.hour.cast("float"))



In [37]:
#number of songs made in each hour
songs_in_hour.show()

+----+-----+
|hour|count|
+----+-----+
|   0| 1709|
|   1| 3546|
|   2| 2094|
|   3|  646|
|  23|  352|
+----+-----+



In [38]:
songs_in_hour_pd = songs_in_hour.toPandas()
songs_in_hour_pd = pd.to_numeric(songs_in_hour_pd.hour)

In [39]:
print(songs_in_hour_pd)

0     0
1     1
2     2
3     3
4    23
Name: hour, dtype: int64


In [46]:
#drop rows with missing values
user_log_valid = user_log.dropna(how = 'any', subset = ['userId', 'sessionId'] )


In [47]:
user_log_valid.count()

10000

downgrade user accounts


In [48]:
user_log_valid.filter("page = 'Submit Downgrade'").show()

+------+---------+---------+------+-------------+--------+------+-----+--------------------+------+----------------+-------------+---------+----+------+-------------+--------------------+------+----+
|artist|     auth|firstName|gender|itemInSession|lastName|length|level|            location|method|            page| registration|sessionId|song|status|           ts|           userAgent|userId|hour|
+------+---------+---------+------+-------------+--------+------+-----+--------------------+------+----------------+-------------+---------+----+------+-------------+--------------------+------+----+
|  null|Logged In|    Kelly|     F|           24|  Newton|  null| paid|Houston-The Woodl...|   PUT|Submit Downgrade|1513283366284|     5931|null|   307|1513768454284|Mozilla/5.0 (Wind...|  1138|   1|
+------+---------+---------+------+-------------+--------+------+-----+--------------------+------+----------------+-------------+---------+----+------+-------------+--------------------+------+----+


In [54]:
user_log.select(["userId", "firstName", "page", "level", "song"]).where(user_log.userId == "1138").collect()


[Row(userId='1138', firstName='Kelly', page='Home', level='paid', song=None),
 Row(userId='1138', firstName='Kelly', page='NextSong', level='paid', song='Everybody Everybody'),
 Row(userId='1138', firstName='Kelly', page='NextSong', level='paid', song='Gears'),
 Row(userId='1138', firstName='Kelly', page='NextSong', level='paid', song='Use Somebody'),
 Row(userId='1138', firstName='Kelly', page='NextSong', level='paid', song='Love Of My Life (1993 Digital Remaster)'),
 Row(userId='1138', firstName='Kelly', page='NextSong', level='paid', song='Down In The Valley Woe'),
 Row(userId='1138', firstName='Kelly', page='NextSong', level='paid', song='Treat Her Like A Lady'),
 Row(userId='1138', firstName='Kelly', page='NextSong', level='paid', song="Everybody Thinks You're An Angel"),
 Row(userId='1138', firstName='Kelly', page='NextSong', level='paid', song='Fourteen Wives'),
 Row(userId='1138', firstName='Kelly', page='NextSong', level='paid', song='Love On The Rocks'),
 Row(userId='1138', f

In [59]:
#signal user account downgrade
flag_downgrade_event = udf(lambda x: 1 if x == "submit downgrade" else 0, IntegerType())
user_log_valid = user_log_valid.withColumn("downgraded", flag_downgrade_event("page"))
user_log_valid.take(5)


[Row(artist='Showaddywaddy', auth='Logged In', firstName='Kenneth', gender='M', itemInSession=112, lastName='Matthews', length=232.93342, level='paid', location='Charlotte-Concord-Gastonia, NC-SC', method='PUT', page='NextSong', registration=1509380319284, sessionId=5132, song='Christmas Tears Will Fall', status=200, ts=1513720872284, userAgent='"Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36"', userId='1046', hour='23', downgraded=0),
 Row(artist='Lily Allen', auth='Logged In', firstName='Elizabeth', gender='F', itemInSession=7, lastName='Chase', length=195.23873, level='free', location='Shreveport-Bossier City, LA', method='PUT', page='NextSong', registration=1512718541284, sessionId=5027, song='Cheryl Tweedy', status=200, ts=1513720878284, userAgent='"Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"', userId='1000', hour='23', downgraded=0),
 Row(artist='Cobra Starship Feat

In [70]:
from pyspark.sql import Window
windowval = Window.partitionBy("userId").orderBy(desc("ts")).rangeBetween(Window.unboundedPreceding, 0)
user_log_valid = user_log_valid.withColumn("phase", Fsum("downgraded").over(windowval))

user_log_valid.select(["userId", "firstName", "ts", "page", "level", "phase"]).where(user_log.userId == "777").sort("ts").collect()



[]

key data insights

In [92]:
user_log.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)
 |-- hour: string (nullable = true)



In [93]:
user_log.head()

Row(artist='Showaddywaddy', auth='Logged In', firstName='Kenneth', gender='M', itemInSession=112, lastName='Matthews', length=232.93342, level='paid', location='Charlotte-Concord-Gastonia, NC-SC', method='PUT', page='NextSong', registration=1509380319284, sessionId=5132, song='Christmas Tears Will Fall', status=200, ts=1513720872284, userAgent='"Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36"', userId='1046', hour='23')

In [91]:
#number of female users
user_log.filter(user_log.gender == "F").select("userId", "gender").dropDuplicates().count()

462

In [94]:
#number of songs played from the favorite artist
user_log.filter(user_log.page == 'NextSong') \
    .select('Artist') \
    .groupBy('Artist') \
    .agg({'Artist':'count'}) \
    .withColumnRenamed('count(Artist)', 'Artistcount') \
    .sort(desc('Artistcount')) \
    .show(1)

+--------+-----------+
|  Artist|Artistcount|
+--------+-----------+
|Coldplay|         83|
+--------+-----------+
only showing top 1 row



In [97]:
from pyspark.sql.functions import isnan, count, when, col, desc, udf, col, sort_array, asc, avg

#number of songs users listen to on average when visitng our home page
function = udf(lambda ishome: int(ishome == 'Home'), IntegerType())
user_window  = Window \
          .partitionBy('userID') \
          .orderBy(desc('ts')) \
          .rangeBetween(Window.unboundedPreceding, 0)

cusum = user_log.filter((user_log.page == "NextSong") | (user_log.page == "Home")) \
      .select('userID', 'page', 'ts') \
      .withColumn('homevisit', function(col('page'))) \
      .withColumn('period', Fsum('homevisit').over(user_window))

cusum.filter((cusum.page == "NextSong")) \
    .groupBy('userID', "period") \
    .agg({"period": "count"}) \
    .agg({"count(period)": "avg"}).show()


+------------------+
|avg(count(period))|
+------------------+
| 6.898347107438017|
+------------------+

